In [69]:
import random
import os
from pydub import AudioSegment, utils
from dotenv import load_dotenv

In [50]:
def split_audio_into_windows(audio, window_duration=10000):
    """
    Takes in a single audio clip and splits it into multiple clips of length `window_duration`.

    Returns:
        list[] windows: a list of audio clips of length `window_duration` in milliseconds.
    """
    windows = []
    audio_duration = len(audio)

    for start_time in range(0, audio_duration, window_duration):
        end_time = start_time + window_duration
        window = audio[start_time:end_time]
        windows.append(window)

    return windows

In [51]:
# Insert the gunshot audio into the background audio at a random interval.
def random_insertion(background_audio, gunshot_audio):
    try:
        # If gunshot_audio can fit, proceed with insertion
        if len(gunshot_audio) <= len(background_audio):
            start_time = random.randint(0, len(background_audio) - len(gunshot_audio))
            result = background_audio.overlay(gunshot_audio, position=start_time)

            return result
    except ValueError as e:
        # Ignore this case and return None
        print(f"Ignoring error: {e}")
        return

VALID_AUDIO_EXTENSIONS = {'.mp3', '.wav', '.flac', '.ogg', '.m4a', '.aac'}

# Create multiple variations of gunshot-inserted audio files.
def create_variations(background_folder, gunshot_folder, output_folder, window_duration=10000):
    background_files = os.listdir(background_folder)
    gunshot_files = os.listdir(gunshot_folder)

    for bg_file_name in background_files:
        bg_file_path = os.path.join(background_folder, bg_file_name)
        bg_audio = AudioSegment.from_file(bg_file_path)

        # Check if the file has a valid audio extension
        if os.path.splitext(bg_file_path)[1].lower() not in VALID_AUDIO_EXTENSIONS:
            continue
        
        for i, window in enumerate(split_audio_into_windows(bg_audio, window_duration)):
            selected_gunshot = random.choice(gunshot_files)
            gunshot_audio = AudioSegment.from_file(os.path.join(gunshot_folder, selected_gunshot))
            
            volume_levels = [0.1, 0.5, 1.0]  # 10% 50% 100%
            for volume in volume_levels:
                # Create a copy of the original gunshot audio with the specified volume level
                modified_gunshot = gunshot_audio.apply_gain(utils.ratio_to_db(volume))

                # Perform the random insertion with the modified gunshot audio
                result_audio = random_insertion(window, modified_gunshot)

                if result_audio is not None:
                    # Save the result audio with a unique name including volume level
                    output_filename = f"{os.path.splitext(bg_file_name)[0]}_window-{i+1}_gun={os.path.splitext(selected_gunshot)[0]}_vol={int(volume * 100)}%.wav"
                    result_audio.export(os.path.join(output_folder, output_filename), format="wav")


In [71]:
load_dotenv(override=True)

# Create variations
create_variations(os.getenv('BACKGROUND_FOLDER'), os.getenv('GUNSHOT_FOLDER'), os.getenv('OUTPUT_FOLDER'))